In [ ]:
# IMPORTS AND MODULE INSTALLATION
import imp, os, subprocess, sys, time, random, pathlib, csv, re
try:
    imp.find_module('selenium')
    imp.find_module('webdriver_manager')
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'selenium']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'webdriver_manager']);
    
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

downloads_folder = str(pathlib.Path.home() / 'Downloads')

def read_config_value(key):
    config = open(downloads_folder + '\\FAST.txt', 'r').read()
    if key + '=' in config:
        return config.split(key + '=')[1].split('\n')[0]
    return 'NULL'

def between(text, start, end):
    try: 
        return text.split(start)[1].split(end)[0]
    except Exception:
        return ''

In [ ]:
%%capture
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install());
driver.minimize_window()

In [ ]:
# VNAS UPLOAD
data_admin_login = 'https://data-admin.virtualnas.net/login'

driver.get(data_admin_login)

driver.find_element('class name', 'btn-success').click()
time.sleep(5)

driver.find_element('id', 'cid') \
    .send_keys(read_config_value('VATSIM_USER'))
driver.find_element('id', 'password') \
    .send_keys(read_config_value('VATSIM_PASS'))
driver.find_element('xpath', \
                    '//button[contains(text(),\'Sign in\')]').click()
time.sleep(2.5)

try:
    ARTCC = read_config_value('ARTCC')
    menu = driver.find_element('class name', 'artcc-menu')
    nav_link = menu.find_element('class name', 'nav-link').click()
    dropdown = menu.find_element('class name', 'dropdown-menu')
    nav_class = str(dropdown.get_attribute('innerHTML')).split(ARTCC)[0] \
        .split('class=\"')[-1].split(' ')[0]
    elems = dropdown.find_elements('class name', nav_class)
    for elem in elems: 
        if ARTCC in elem.text: elem.click()
    time.sleep(2.5)
except Exception:
    pass

In [ ]:
data_admin_url = 'https://data-admin.virtualnas.net/training/scenarios/'
driver.get(data_admin_url + read_config_value('SCENARIO'))

driver.execute_script("window.scrollBy(0, 500);")

def get_plane_id():
    return re.findall(r'aircraft\[[0-9]{1,4}\]', driver.page_source)[0] \
    .split('[')[1].split(']')[0]
    
def set_plane_data(pos, element, value):
    driver.find_element('name', 'aircraft[' + pos + '].' 
                        + element).clear()
    driver.find_element('name', 'aircraft[' + pos + '].' 
                        + element).send_keys(value)
    
def set_plane_data_list(pos, element, value):
    webdriver.support.ui.Select(
        driver.find_element('name', 'aircraft[' + pos + '].' + element)) \
    .select_by_value(value)

with open(downloads_folder + '\\' + 
          read_config_value('CSV_FILE'), 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for plane in reader:
        driver.find_element(,"//button[text()='Add Aircraft']") \
            .click() # ERROR
        pos = get_plane_id()
        set_plane_data(pos, 'aircraftId', plane['ident'])
        set_plane_data(pos, 'aircraftType', plane['type'])
        set_plane_data_list(pos, 'transponderMode', 'Standby')
        true_spawn_delay = round(int(plane['spawn-delay']) \
            / float(read_config_value('TIME_COMPRESSION')) \
            - int(read_config_value('TIME_OFFSET')))
        if true_spawn_delay < 0: true_spawn_delay = 0
        print(true_spawn_delay)
        set_plane_data(pos, 'spawnDelay', true_spawn_delay)
        set_plane_data(pos, 'airportId', plane['dep'][1:])

        try:
            driver.find_element('xpath', 
                '//button[contains(text(),\'Create Flight Plan\')]') \
                .click()
        except Exception:
            pass
        
        set_plane_data(pos, 'flightplan.departure', plane['dep'])
        set_plane_data(pos, 'flightplan.destination', plane['arr'])
        set_plane_data(pos, 'flightplan.cruiseAltitude', plane['alt'])
        set_plane_data(pos, 'flightplan.cruiseSpeed', plane['speed'])
        set_plane_data(pos, 'flightplan.aircraftType', plane['type'] 
                       + '/L')
        set_plane_data(pos, 'flightplan.route', plane['route'])
        set_plane_data(pos, 'flightplan.remarks', '/v/')
        webdriver.support.ui.Select(driver.find_element('xpath', 
            '//option[text()=\'Coordinates\']') \
            .find_element('xpath', '..')).select_by_value('Parking')
        set_plane_data(pos, 'startingConditions.parking', plane['gate'])

        driver.find_element('xpath', 
                '//button[contains(text(),\'Done\')]').click()